In [3]:
import wrds
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import time
from pandas.tseries.offsets import MonthEnd
import locale
locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' )

'en_US.UTF-8'

In [4]:
db = wrds.Connection()

/Users/townes/anaconda/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [5]:
def first_trading_day(date): #If anyone can think of a better way of doing this, by all means implement it
    dow = date.dayofweek #Monday = 0, Sunday = 6
                            
    if dow in [5, 6]:
        date = date + pd.Timedelta( str(7 - dow) + ' days')
            
    if dow == 0:
        date = date + pd.Timedelta(str(1) + ' days')
    
    if date.year == 2007: #Day of mourning for G . Ford on this year
        date = date + pd.Timedelta(str(1) + ' days')
    
    return date.strftime('%Y-%m-%d')

In [21]:
window_size = 3   # window size in years

In [6]:
year = 2001
daily_start_date = first_trading_day(pd.to_datetime('1-2-' + str(year))) 

print(daily_start_date)

2001-01-02


In [7]:
monthly_start_date = "{}-01-31".format(str(year))
print(monthly_start_date)

2001-01-31


# 1

In [8]:
marketcap_df = pd.DataFrame()

In [9]:
start_date_pd = pd.to_datetime(daily_start_date)
date = [start_date_pd + MonthEnd(0) - pd.Timedelta(str(10) + ' days'), start_date_pd + MonthEnd(1)] 
date_str = [date[0].strftime('%Y-%m-%d'),date[1].strftime('%Y-%m-%d')]

date_str

['2001-01-21', '2001-01-31']

In [10]:
statement = "select all cusip, shrout, prc from crspq.msf where date > '{}' and date <= '{}'" . format(date_str[0], date_str[1])

print(statement)

select all cusip, shrout, prc from crspq.msf where date > '2001-01-21' and date <= '2001-01-31'


In [11]:
marketcap_df = db.raw_sql(statement)

In [12]:
print(marketcap_df)

         cusip     shrout        prc
0     36720410     2498.0   9.875000
1     05978R10     8526.0  10.000000
2     23326910    20897.0   3.000000
3     81002230    16964.0  23.062500
4     44950710     8270.0   3.750000
5     00103110     7655.0  37.625000
6     46603210     8427.0  15.312500
7     23323G10     4908.0   9.500000
8     72913210    41067.0  46.937500
9     38209110    22867.0   6.062500
10    35242V10     3553.0  12.250000
11    00208K10     5418.0  13.625000
12    13886930    11624.0   0.940000
13    77467X10     1961.0   5.000000
14    41043F20    18910.0   1.310000
15    65163310     4591.0  11.562500
16    00621210    79186.0  21.870001
17    35472720    39467.0   0.260000
18    86681020  3259868.0  30.562500
19    61576210     1819.0  -1.093750
20    90273710    46817.0   8.250000
21    02078110     7078.0   9.750000
22    35086510    12080.0  14.200000
23    02713510     4274.0  -2.531250
24    68389X10  5585918.0  29.125000
25    59491810  5335391.0  61.062500
2

In [13]:
marketcap_df["market_cap"] = marketcap_df.shrout * abs(marketcap_df.prc)

print(marketcap_df)

         cusip     shrout        prc    market_cap
0     36720410     2498.0   9.875000  2.466775e+04
1     05978R10     8526.0  10.000000  8.526000e+04
2     23326910    20897.0   3.000000  6.269100e+04
3     81002230    16964.0  23.062500  3.912322e+05
4     44950710     8270.0   3.750000  3.101250e+04
5     00103110     7655.0  37.625000  2.880194e+05
6     46603210     8427.0  15.312500  1.290384e+05
7     23323G10     4908.0   9.500000  4.662600e+04
8     72913210    41067.0  46.937500  1.927582e+06
9     38209110    22867.0   6.062500  1.386312e+05
10    35242V10     3553.0  12.250000  4.352425e+04
11    00208K10     5418.0  13.625000  7.382025e+04
12    13886930    11624.0   0.940000  1.092656e+04
13    77467X10     1961.0   5.000000  9.805000e+03
14    41043F20    18910.0   1.310000  2.477210e+04
15    65163310     4591.0  11.562500  5.308344e+04
16    00621210    79186.0  21.870001  1.731798e+06
17    35472720    39467.0   0.260000  1.026142e+04
18    86681020  3259868.0  30.5

In [14]:
marketcap_df_sorted = marketcap_df.sort_values(by=['market_cap'])

print(marketcap_df_sorted)

         cusip   shrout        prc   market_cap
1025  89290970    148.0   0.187500    27.750000
891   23379840     10.0  -3.500000    35.000000
7413  15133C50     10.0  -6.875000    68.750000
7874  28225R20     17.0  -9.687500   164.687500
7609  64378H10    940.0   0.260000   244.399991
5299  04940820    123.0   2.500000   307.500000
5001  80601810     20.0 -23.850000   477.000008
1186  43123220    287.0  -1.843750   529.156250
7050  35687210     29.0  20.000000   580.000000
5000  74050U10     61.0  11.490000   700.889986
7479  45795510    203.0   3.750000   761.250000
4297  26928220    625.0  -1.250000   781.250000
6164  25534010   4256.0   0.187500   798.000000
7838  29349110     80.0 -10.350000   828.000031
7434  87425410   3035.0   0.281250   853.593750
6279  70415720   2403.0   0.375000   901.125000
2802  85515630    589.0   1.687500   993.937500
6046  12596130    842.0   1.343750  1131.437500
6854  87295810   2200.0   0.520000  1143.999958
1787  71673220   1059.0   1.125000  1191

In [15]:
marketcap_df_sorted_capped = marketcap_df_sorted.copy()[marketcap_df_sorted.market_cap.notnull()]

print(marketcap_df_sorted_capped)

         cusip     shrout           prc    market_cap
1025  89290970      148.0      0.187500  2.775000e+01
891   23379840       10.0     -3.500000  3.500000e+01
7413  15133C50       10.0     -6.875000  6.875000e+01
7874  28225R20       17.0     -9.687500  1.646875e+02
7609  64378H10      940.0      0.260000  2.444000e+02
5299  04940820      123.0      2.500000  3.075000e+02
5001  80601810       20.0    -23.850000  4.770000e+02
1186  43123220      287.0     -1.843750  5.291562e+02
7050  35687210       29.0     20.000000  5.800000e+02
5000  74050U10       61.0     11.490000  7.008900e+02
7479  45795510      203.0      3.750000  7.612500e+02
4297  26928220      625.0     -1.250000  7.812500e+02
6164  25534010     4256.0      0.187500  7.980000e+02
7838  29349110       80.0    -10.350000  8.280000e+02
7434  87425410     3035.0      0.281250  8.535938e+02
6279  70415720     2403.0      0.375000  9.011250e+02
2802  85515630      589.0      1.687500  9.939375e+02
6046  12596130      842.0   

In [16]:
marketcap_df_sorted_capped["market_cap_cum"] = marketcap_df_sorted_capped.market_cap.cumsum()

print(marketcap_df_sorted_capped)

         cusip     shrout           prc    market_cap  market_cap_cum
1025  89290970      148.0      0.187500  2.775000e+01    2.775000e+01
891   23379840       10.0     -3.500000  3.500000e+01    6.275000e+01
7413  15133C50       10.0     -6.875000  6.875000e+01    1.315000e+02
7874  28225R20       17.0     -9.687500  1.646875e+02    2.961875e+02
7609  64378H10      940.0      0.260000  2.444000e+02    5.405875e+02
5299  04940820      123.0      2.500000  3.075000e+02    8.480875e+02
5001  80601810       20.0    -23.850000  4.770000e+02    1.325087e+03
1186  43123220      287.0     -1.843750  5.291562e+02    1.854244e+03
7050  35687210       29.0     20.000000  5.800000e+02    2.434244e+03
5000  74050U10       61.0     11.490000  7.008900e+02    3.135134e+03
7479  45795510      203.0      3.750000  7.612500e+02    3.896384e+03
4297  26928220      625.0     -1.250000  7.812500e+02    4.677634e+03
6164  25534010     4256.0      0.187500  7.980000e+02    5.475634e+03
7838  29349110      

In [17]:
marketcap_df_sorted_capped["quantile"] = marketcap_df_sorted_capped["market_cap_cum"].transform(lambda x: pd.cut(x, 40, labels=range(1,41)))

print(marketcap_df_sorted_capped)

         cusip     shrout           prc    market_cap  market_cap_cum quantile
1025  89290970      148.0      0.187500  2.775000e+01    2.775000e+01        1
891   23379840       10.0     -3.500000  3.500000e+01    6.275000e+01        1
7413  15133C50       10.0     -6.875000  6.875000e+01    1.315000e+02        1
7874  28225R20       17.0     -9.687500  1.646875e+02    2.961875e+02        1
7609  64378H10      940.0      0.260000  2.444000e+02    5.405875e+02        1
5299  04940820      123.0      2.500000  3.075000e+02    8.480875e+02        1
5001  80601810       20.0    -23.850000  4.770000e+02    1.325087e+03        1
1186  43123220      287.0     -1.843750  5.291562e+02    1.854244e+03        1
7050  35687210       29.0     20.000000  5.800000e+02    2.434244e+03        1
5000  74050U10       61.0     11.490000  7.008900e+02    3.135134e+03        1
7479  45795510      203.0      3.750000  7.612500e+02    3.896384e+03        1
4297  26928220      625.0     -1.250000  7.812500e+0

In [76]:
smallcap_df = marketcap_df_sorted_capped.loc[marketcap_df_sorted_capped['quantile'].isin(range(6))]
smallcap_df.reset_index(inplace=True)
print(smallcap_df)

      index     cusip    shrout         prc    market_cap  market_cap_cum  \
0      1025  89290970     148.0    0.187500  2.775000e+01    2.775000e+01   
1       891  23379840      10.0   -3.500000  3.500000e+01    6.275000e+01   
2      7413  15133C50      10.0   -6.875000  6.875000e+01    1.315000e+02   
3      7874  28225R20      17.0   -9.687500  1.646875e+02    2.961875e+02   
4      7609  64378H10     940.0    0.260000  2.444000e+02    5.405875e+02   
5      5299  04940820     123.0    2.500000  3.075000e+02    8.480875e+02   
6      5001  80601810      20.0  -23.850000  4.770000e+02    1.325087e+03   
7      1186  43123220     287.0   -1.843750  5.291562e+02    1.854244e+03   
8      7050  35687210      29.0   20.000000  5.800000e+02    2.434244e+03   
9      5000  74050U10      61.0   11.490000  7.008900e+02    3.135134e+03   
10     7479  45795510     203.0    3.750000  7.612500e+02    3.896384e+03   
11     4297  26928220     625.0   -1.250000  7.812500e+02    4.677634e+03   

In [19]:
smallcap_cusip = smallcap_df["cusip"]

print(smallcap_cusip)

0       89290970
1       23379840
2       15133C50
3       28225R20
4       64378H10
5       04940820
6       80601810
7       43123220
8       35687210
9       74050U10
10      45795510
11      26928220
12      25534010
13      29349110
14      87425410
15      70415720
16      85515630
17      12596130
18      87295810
19      71673220
20      67106210
21      13981810
22      51462430
23      36230D20
24      88579310
25      74372Q20
26      44107210
27      92924P20
28      80601820
29      42197910
          ...   
7156    85809810
7157    25365110
7158    91913Y10
7159    H2906T10
7160    15640T10
7161    12653C10
7162    83092810
7163    66728010
7164    00483310
7165    74531010
7166    91911X10
7167    29470070
7168    74790650
7169    45727710
7170    50544710
7171    90254950
7172    73172K10
7173    33747710
7174    21775310
7175    27826510
7176    46261310
7177    30249130
7178    31190010
7179    60365K10
7180    68628610
7181    89618L10
7182    80004C10
7183    170388

# 222222222222

In [284]:
test_cusip = smallcap_cusip[0]

print(test_cusip)

89290970


In [314]:
test_qtr = 4

In [315]:
test_statement = "select a.cusip, b.conm, b.tic, b.rdq, b.atq, b.dlttq, b.ltq, b.cheq, b.oibdpq, b.ppentq, b.piq, b.revtq, b.niq, b.cogsq, b.xoprq, b.xintq, b.oiadpq, b.dlcq from crspq.dsf a join comp.fundq b on b.cusip::varchar(8) = a.cusip where a.date = '" + daily_start_date + "' and b.fyearq = " + str(year) + " and b.fqtr = " + str(test_qtr) 

print(test_statement)

select a.cusip, b.conm, b.tic, b.rdq, b.atq, b.dlttq, b.ltq, b.cheq, b.oibdpq, b.ppentq, b.piq, b.revtq, b.niq, b.cogsq, b.xoprq, b.xintq, b.oiadpq, b.dlcq from crspq.dsf a join comp.fundq b on b.cusip::varchar(8) = a.cusip where a.date = '2001-01-02' and b.fyearq = 2001 and b.fqtr = 4


In [316]:
test_smallcap_df = db.raw_sql(test_statement)

In [317]:
print(test_smallcap_df.shape)

(6518, 18)


In [318]:
test_cusip = '23379840'

In [319]:
test_smallcap_df.loc[test_smallcap_df["cusip"] == test_cusip].shape

(0, 18)

In [320]:
valid_comp_list = list()

for cusip in smallcap_cusip:
    comp_info = test_smallcap_df.loc[test_smallcap_df["cusip"] == cusip]
    if comp_info.shape[0] != 0:
        valid_comp_list.append(cusip)
        
#print(valid_comp_list)

In [321]:
print(valid_comp_list)

4761


# 333333

In [203]:
test_cusip = valid_comp_list[0]

print(test_cusip)

64378H10


In [208]:
test_statement = "select b.conm, b.tic, b.rdq, b.atq, b.dlttq, b.ltq, b.cheq, b.oibdpq, b.ppentq, b.piq, b.revtq, b.niq, b.cogsq, b.xoprq, b.xintq, b.oiadpq, b.dlcq from crspq.dsf a join comp.fundq b on b.cusip::varchar(8) = a.cusip where a.cusip = '{}' and a.date = '{}' and b.fyearq = {} and b.fqtr = {}".format(test_cusip, daily_start_date, year, test_qtr)

print(test_statement)

select b.conm, b.tic, b.rdq, b.atq, b.dlttq, b.ltq, b.cheq, b.oibdpq, b.ppentq, b.piq, b.revtq, b.niq, b.cogsq, b.xoprq, b.xintq, b.oiadpq, b.dlcq from crspq.dsf a join comp.fundq b on b.cusip::varchar(8) = a.cusip where a.cusip = '64378H10' and a.date = '2001-01-02' and b.fyearq = 2001 and b.fqtr = 3


In [205]:
test_singlecomp_df = db.raw_sql(test_statement)

In [206]:
print(test_singlecomp_df)

                   conm   tic         rdq  atq  dlttq  ltq  cheq  oibdpq  \
0  NEW DRAGON ASIA CORP  NDAC  2001-11-15  0.0    0.0  0.0   0.0   1.093   

   ppentq    piq  revtq  niq  cogsq  xoprq xintq  oiadpq  dlcq  
0     0.0  0.773  7.713  0.7  6.129   6.62  None   0.791   0.0  


## Select list of info from several cusip

In [280]:
cusip_list = ", ".join(["'{}'".format(x) for x in valid_comp_list])

#print(cusip_list)

In [281]:
test_statement = "select b.conm, b.tic, b.rdq, b.atq, b.dlttq, b.ltq, b.cheq, b.oibdpq, b.ppentq, b.piq, b.revtq, b.niq, b.cogsq, b.xoprq, b.xintq, b.oiadpq, b.dlcq from crspq.dsf a join comp.fundq b on b.cusip::varchar(8) = a.cusip where a.cusip in ({}) and a.date = '{}' and b.fyearq = {} and b.fqtr = {}".format(cusip_list, daily_start_date, year, test_qtr)

#print(test_statement)

In [282]:
test_lotsofcomp_df = db.raw_sql(test_statement)

In [283]:
print(test_lotsofcomp_df.shape)

(4958, 17)


In [264]:
test_lotsofcomp_df_notnull = test_lotsofcomp_df.copy()

In [265]:
index_list = ["conm", "tic", "rdq", "atq", "dlttq", "ltq", "cheq", "oibdpq", "ppentq", "piq", "revtq", "niq", "cogsq", \
              "xoprq", "xintq", "oiadpq", "dlcq"]

for index in index_list:
    test_lotsofcomp_df_notnull = test_lotsofcomp_df_notnull[test_lotsofcomp_df_notnull[index].notnull()]
    
print(test_lotsofcomp_df_notnull.shape)

(2284, 17)


In [267]:
test_lotsofcomp_df_notnull.reset_index()

,index,conm,tic,rdq,atq,dlttq,ltq,cheq,oibdpq,ppentq,piq,revtq,niq,cogsq,xoprq,xintq,oiadpq,dlcq
0,1,AEP INDUSTRIES INC,AEPI,2001-09-05,436.059,229.673,376.797,2.554,10.723,193.617,-0.934,157.554,-0.551,122.286,146.831,6.594,3.861,26.344
1,2,AHL SERVICES INC,AHLS,2001-11-13,275.188,0.000,144.194,3.852,-22.528,21.354,-33.296,32.791,-70.617,14.704,55.319,1.101,-32.195,98.312
2,3,AHPC HOLDINGS INC,GLOV,2001-05-11,38.958,0.017,16.677,0.510,0.402,12.350,-0.216,12.461,-0.064,8.671,12.059,0.224,-0.100,6.924
3,4,AMAG PHARMACEUTICALS INC,AMAG,2001-07-19,33.150,0.000,17.378,28.178,0.327,3.865,0.730,1.471,0.704,1.144,1.144,0.000,0.197,0.000
4,5,AMC ENTERTAINMENT INC -OLD,AEN.2,2002-01-22,1065.708,541.296,897.506,48.428,29.244,770.634,-10.800,316.753,-9.000,274.925,287.509,13.990,2.944,2.629
5,6,AMX CORP,AMXC,2002-01-31,38.625,0.000,25.254,1.134,1.951,8.943,0.286,21.286,0.299,10.005,19.335,0.154,0.990,10.565
6,7,ANFI INC,ANFI.1,2001-10-25,62.038,4.706,29.005,12.617,5.661,10.502,4.892,33.738,2.837,28.077,28.077,0.102,4.994,0.806
7,8,AT PLASTICS INC,ATPIF,2001-10-30,467.736,0.000,254.229,13.744,9.704,326.905,-12.000,62.323,-8.475,52.619,52.619,6.449,3.558,206.893
8,11,AXS-ONE INC,3AXSO,2001-11-02,10.989,1.799,20.015,0.671,1.050,0.890,0.487,9.821,0.487,3.873,8.771,0.152,0.548,1.800
9,12,AZZ INC,AZZ,2001-12-19,143.453,53.095,91.322,1.487,5.560,36.624,3.284,35.257,2.035,25.521,29.697,0.539,3.933,10.040


## Pull 3 years 12 quaters financials

In [349]:
quaters = ", ".join(["'{}'".format(x) for x in [1, 2, 3, 4]])
cusip_list = ", ".join(["'{}'".format(x) for x in valid_comp_list[100:103]])
years = ", ".join(["{}".format(y) for y in range(year, year + 3)])

print(quaters)
print(cusip_list)
print(years)

'1', '2', '3', '4'
'56660510', '46428677', '00485720'
2001, 2002, 2003


In [352]:
test_statement = "select a.cusip, b.conm, b.tic, b.rdq, b.atq, b.dlttq, b.ltq, b.cheq, b.oibdpq, b.ppentq, b.piq, b.revtq, b.niq, b.cogsq, b.xoprq, b.xintq, b.oiadpq, b.dlcq from crspq.dsf a join comp.fundq b on b.cusip::varchar(8) = a.cusip where a.cusip in ({}) and a.date = '{}' and b.fyearq in ({}) and b.fqtr in ({})".format(cusip_list, daily_start_date, years, quaters)

print(test_statement)

select a.cusip, b.conm, b.tic, b.rdq, b.atq, b.dlttq, b.ltq, b.cheq, b.oibdpq, b.ppentq, b.piq, b.revtq, b.niq, b.cogsq, b.xoprq, b.xintq, b.oiadpq, b.dlcq from crspq.dsf a join comp.fundq b on b.cusip::varchar(8) = a.cusip where a.cusip in ('56660510', '46428677', '00485720') and a.date = '2001-01-02' and b.fyearq in (2001, 2002, 2003) and b.fqtr in ('1', '2', '3', '4')


## Test Merge

In [449]:
smallcap_cusip = smallcap_df[['cusip']]
merged_df = smallcap_cusip.copy()

#merged_df

In [450]:
# Obtain data from quater 1

quater = 1

print(daily_start_date)
print(year)
print(quater)

2001-01-02
2001
1


In [451]:
test_statement = "select a.cusip, b.conm, b.tic, b.rdq, b.atq, b.dlttq, b.ltq, b.cheq, b.oibdpq, b.ppentq, b.piq, b.revtq, b.niq, b.cogsq, b.xoprq, b.xintq, b.oiadpq, b.dlcq from crspq.dsf a join comp.fundq b on b.cusip::varchar(8) = a.cusip where a.date = '{}' and b.fyearq = {} and b.fqtr = {}".format(daily_start_date, year, quater)

print(test_statement)

select a.cusip, b.conm, b.tic, b.rdq, b.atq, b.dlttq, b.ltq, b.cheq, b.oibdpq, b.ppentq, b.piq, b.revtq, b.niq, b.cogsq, b.xoprq, b.xintq, b.oiadpq, b.dlcq from crspq.dsf a join comp.fundq b on b.cusip::varchar(8) = a.cusip where a.date = '2001-01-02' and b.fyearq = 2001 and b.fqtr = 1


In [452]:
test_df_1 = db.raw_sql(test_statement)

In [453]:
# Label the columns with the quater. Create a label mapping to rename column labels
label_map = {x : x + "_" + str(quater) for x in test_df_1.columns if x not in ['cusip', 'conm', 'tic']}

label_map

{'atq': 'atq_1',
 'cheq': 'cheq_1',
 'cogsq': 'cogsq_1',
 'dlcq': 'dlcq_1',
 'dlttq': 'dlttq_1',
 'ltq': 'ltq_1',
 'niq': 'niq_1',
 'oiadpq': 'oiadpq_1',
 'oibdpq': 'oibdpq_1',
 'piq': 'piq_1',
 'ppentq': 'ppentq_1',
 'rdq': 'rdq_1',
 'revtq': 'revtq_1',
 'xintq': 'xintq_1',
 'xoprq': 'xoprq_1'}

In [454]:
# Rename the column labels
test_df_1_renamed = test_df_1.rename(columns=label_map)

test_df_1_renamed.shape

(7071, 18)

In [455]:
# Merge dataframe
merged_df = pd.merge(merged_df, test_df_1_renamed, how='inner', on=['cusip'])

merged_df

,cusip,conm,tic,rdq_1,atq_1,dlttq_1,ltq_1,cheq_1,oibdpq_1,ppentq_1,piq_1,revtq_1,niq_1,cogsq_1,xoprq_1,xintq_1,oiadpq_1,dlcq_1
0,64378H10,NEW DRAGON ASIA CORP,NDAC,2001-07-10,3.319,0.000,4.101,0.038,0.999,0.590,0.650,7.387,0.594,5.716,6.388,NaN,0.759,2.274
1,04940820,ATLAS PEARLS LTD,APCFY,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,80601810,SCANIA AB,SCVAY,2001-05-30,5236.871,1070.517,3653.237,277.449,142.094,2410.878,78.845,1200.963,55.162,1058.869,1058.869,NaN,91.071,NaN
3,74050U10,PREMIER FARNELL PLC,PFP.1,2001-05-24,809.300,316.500,634.900,98.200,NaN,153.600,14.700,309.200,5.200,273.800,273.800,4.900,35.400,0.000
4,25534010,DIVERSIFIED SENIOR SERVICES,DISS,2001-05-31,10.266,0.625,2.917,0.011,-0.168,0.078,-0.135,1.675,-0.135,1.843,1.843,NaN,-0.181,0.375
5,29349110,ENODIS PLC,ENO.1,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,85515630,STAR MULTI CARE SERVICES,3SMCS,2001-10-22,10.810,0.000,6.864,0.032,0.183,0.800,-0.532,7.984,1.075,5.373,7.801,0.448,-0.063,1.100
7,12596130,CTI INDUSTRIES CORP,CTIB,2001-05-15,23.715,4.579,18.893,0.435,0.569,8.928,-0.155,6.081,-0.140,4.068,5.512,0.307,0.178,6.236
8,87295810,TPN HOLDINGS PLC -ADR,TPNHY,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,71673220,PETROMINERALS CORP,PTRO,2001-05-15,2.260,0.000,0.083,1.758,-0.105,0.378,-0.106,0.155,-0.106,0.122,0.260,0.000,-0.106,0.000


In [456]:
merged_df.shape

(5214, 18)

In [457]:
quater = 2
test_statement = "select a.cusip, b.conm, b.tic, b.rdq, b.atq, b.dlttq, b.ltq, b.cheq, b.oibdpq, b.ppentq, b.piq, b.revtq, b.niq, b.cogsq, b.xoprq, b.xintq, b.oiadpq, b.dlcq from crspq.dsf a join comp.fundq b on b.cusip::varchar(8) = a.cusip where a.date = '{}' and b.fyearq = {} and b.fqtr = {}".format(daily_start_date, year, quater)

print(test_statement)

select a.cusip, b.conm, b.tic, b.rdq, b.atq, b.dlttq, b.ltq, b.cheq, b.oibdpq, b.ppentq, b.piq, b.revtq, b.niq, b.cogsq, b.xoprq, b.xintq, b.oiadpq, b.dlcq from crspq.dsf a join comp.fundq b on b.cusip::varchar(8) = a.cusip where a.date = '2001-01-02' and b.fyearq = 2001 and b.fqtr = 2


In [458]:
test_df_2 = db.raw_sql(test_statement)

In [460]:
test_df_2.shape

(6912, 18)

In [463]:
# Label the columns with the quater. Create a label mapping to rename column labels
label_map = {x : x + "_" + str(quater) for x in test_df_1.columns if x not in ['cusip', 'conm', 'tic']}

label_map

{'atq': 'atq_2',
 'cheq': 'cheq_2',
 'cogsq': 'cogsq_2',
 'dlcq': 'dlcq_2',
 'dlttq': 'dlttq_2',
 'ltq': 'ltq_2',
 'niq': 'niq_2',
 'oiadpq': 'oiadpq_2',
 'oibdpq': 'oibdpq_2',
 'piq': 'piq_2',
 'ppentq': 'ppentq_2',
 'rdq': 'rdq_2',
 'revtq': 'revtq_2',
 'xintq': 'xintq_2',
 'xoprq': 'xoprq_2'}

In [464]:
# Rename the column labels
test_df_2_renamed = test_df_2.rename(columns=label_map)

test_df_2_renamed

,cusip,conm,tic,rdq_2,atq_2,dlttq_2,ltq_2,cheq_2,oibdpq_2,ppentq_2,piq_2,revtq_2,niq_2,cogsq_2,xoprq_2,xintq_2,oiadpq_2,dlcq_2
0,00036020,AAON INC,AAON,2001-07-17,84.890,5.349,40.628,0.013,7.288,34.492,6.036,41.520,3.816,29.563,34.232,0.312,6.213,9.593
1,00036110,AAR CORP,AIR,2001-12-20,714.208,189.733,429.864,53.129,5.513,105.513,-90.655,144.889,-54.484,118.697,139.376,5.426,-0.171,71.162
2,00075210,ABC-NACO INC,ABCRQ,2001-08-08,393.813,10.166,363.071,1.260,-2.998,208.324,-27.736,97.426,-24.480,89.591,100.424,7.728,-12.127,227.835
3,00078810,A B WATLEY GROUP INC,ABWG,2001-05-15,25.425,8.674,19.984,2.522,-3.049,17.643,-4.608,7.163,-4.608,10.212,10.212,0.239,-4.335,0.011
4,00086T10,A.C. MOORE ARTS & CRAFTS INC,ACMR.1,2001-07-19,114.052,2.540,50.588,4.016,0.998,24.000,-0.374,67.132,-0.232,41.042,66.134,0.149,-0.225,18.760
5,00087B10,ACR GROUP INC,BRR,2001-10-15,60.894,2.450,49.307,0.267,2.064,5.676,1.348,46.509,1.044,36.588,44.445,0.550,1.758,23.135
6,00088630,ADC TELECOMMUNICATIONS INC,ADCT,2001-05-24,2972.300,10.300,828.800,432.300,-112.600,724.900,-1459.200,652.400,-1045.200,463.200,765.000,NaN,-168.300,124.400
7,00088E10,ACTV INC,IATV,2001-08-02,184.652,0.000,80.399,86.827,-4.310,17.099,7.308,4.227,8.128,8.537,8.537,0.000,-6.819,0.000
8,00088U10,A D A M INC,ADAM.1,2001-08-01,8.075,0.000,3.086,3.771,0.414,0.619,1.744,2.395,1.717,0.268,1.981,NaN,0.272,0.000
9,00089C10,ADE CORP/MA,ADEX,2001-11-27,127.259,11.028,38.007,32.865,-2.781,29.173,-3.969,21.910,-14.567,12.384,24.691,NaN,-4.127,0.633


In [465]:
merged_df = pd.merge(merged_df, test_df_2_renamed, how='inner', on=['cusip'])

merged_df

,cusip,conm_x,tic_x,rdq_1,atq_1,dlttq_1,ltq_1,cheq_1,oibdpq_1,ppentq_1,...,oibdpq_2,ppentq_2,piq_2,revtq_2,niq_2,cogsq_2,xoprq_2,xintq_2,oiadpq_2,dlcq_2
0,64378H10,NEW DRAGON ASIA CORP,NDAC,2001-07-10,3.319,0.000,4.101,0.038,0.999,0.590,...,1.040,0.549,0.760,5.917,0.713,4.603,4.877,NaN,0.773,2.288
1,04940820,ATLAS PEARLS LTD,APCFY,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,80601810,SCANIA AB,SCVAY,2001-05-30,5236.871,1070.517,3653.237,277.449,142.094,2410.878,...,-11.499,2371.381,33.828,1206.261,21.853,1217.760,1217.760,NaN,-58.353,NaN
3,74050U10,PREMIER FARNELL PLC,PFP.1,2001-05-24,809.300,316.500,634.900,98.200,NaN,153.600,...,NaN,160.600,22.800,272.100,15.800,244.300,244.300,5.000,27.800,252.300
4,25534010,DIVERSIFIED SENIOR SERVICES,DISS,2001-05-31,10.266,0.625,2.917,0.011,-0.168,0.078,...,-0.106,0.125,-0.772,1.588,-0.772,1.694,1.694,NaN,-0.128,0.509
5,29349110,ENODIS PLC,ENO.1,None,NaN,NaN,NaN,NaN,NaN,NaN,...,0.852,244.919,-55.767,785.984,-59.314,785.132,785.132,NaN,-33.630,0.568
6,85515630,STAR MULTI CARE SERVICES,3SMCS,2001-10-22,10.810,0.000,6.864,0.032,0.183,0.800,...,-0.704,0.728,-0.887,2.370,-0.887,1.565,3.074,0.085,-0.810,0.900
7,12596130,CTI INDUSTRIES CORP,CTIB,2001-05-15,23.715,4.579,18.893,0.435,0.569,8.928,...,0.567,8.915,0.067,6.876,0.040,4.739,6.309,0.262,0.238,6.397
8,87295810,TPN HOLDINGS PLC -ADR,TPNHY,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.192,-0.013,NaN,NaN,NaN,NaN,NaN
9,71673220,PETROMINERALS CORP,PTRO,2001-05-15,2.260,0.000,0.083,1.758,-0.105,0.378,...,-0.115,0.380,-0.117,0.156,-0.117,0.134,0.271,0.000,-0.116,0.000


In [466]:
merged_df.shape

(5108, 35)

# Use a for-lopp to join all dataframe of 12 consecutive quaters from crspq.dsf & comp.fundq

In [57]:
# Initialize merged_df 

smallcap_cusip = smallcap_df[['cusip']]
merged_fin_df = smallcap_cusip.copy()

In [58]:


for y in range(year, year + window_size):
    for q in [1, 2, 3, 4]:
        print('year: %d, quater: %d' % (y, q))
        
        # Getting raw data
        daily_start_date = first_trading_day(pd.to_datetime('1-2-' + str(y))) 
        statement = "select a.cusip, b.conm, b.tic, b.rdq, b.atq, b.dlttq, b.ltq, b.cheq, b.oibdpq, b.ppentq, b.piq, b.revtq, b.niq, b.cogsq, b.xoprq, b.xintq, b.oiadpq, b.dlcq from crspq.dsf a join comp.fundq b on b.cusip::varchar(8) = a.cusip where a.date = '{}' and b.fyearq = {} and b.fqtr = {}".format(daily_start_date, y, q)
        financials_qtr_df = db.raw_sql(statement)
        
        # Create the label mapping to rename column labels
        label_map = {x : x + "_" + str(q + (y-year) * 4) for x in financials_qtr_df.columns if x not in ['cusip', 'conm', 'tic']}
        
        # Use label mapping to rename column labels
        financials_qtr_df.rename(columns=label_map, inplace=True)
        
        # Merge dataframe according to values in 'cusip' column
        if 'conm' and 'tic' not in merged_fin_df.columns:
            merged_fin_df = pd.merge(merged_fin_df, financials_qtr_df, how='inner', on=['cusip'])
        else:
            merged_fin_df = pd.merge(merged_fin_df, financials_qtr_df, how='inner', on=['cusip', 'conm', 'tic'])

year: 2001, quater: 1
year: 2001, quater: 2
year: 2001, quater: 3
year: 2001, quater: 4
year: 2002, quater: 1
year: 2002, quater: 2
year: 2002, quater: 3
year: 2002, quater: 4
year: 2003, quater: 1
year: 2003, quater: 2
year: 2003, quater: 3
year: 2003, quater: 4


In [59]:
merged_fin_df

,cusip,conm,tic,rdq_1,atq_1,dlttq_1,ltq_1,cheq_1,oibdpq_1,ppentq_1,...,oibdpq_12,ppentq_12,piq_12,revtq_12,niq_12,cogsq_12,xoprq_12,xintq_12,oiadpq_12,dlcq_12
0,64378H10,NEW DRAGON ASIA CORP,NDAC,2001-07-10,3.319,0.000,4.101,0.038,0.999,0.590,...,1.743,21.469,2.342,10.553,2.213,6.975,8.810,0.054,1.384,2.845
1,04940820,ATLAS PEARLS LTD,APCFY,None,NaN,NaN,NaN,NaN,NaN,NaN,...,0.905,1.848,0.543,2.986,0.327,2.081,2.081,0.000,0.281,0.000
2,74050U10,PREMIER FARNELL PLC,PFP.1,2001-05-24,809.300,316.500,634.900,98.200,NaN,153.600,...,NaN,195.265,31.800,402.319,24.580,365.748,365.748,7.395,36.571,5.282
3,29349110,ENODIS PLC,ENO.1,None,NaN,NaN,NaN,NaN,NaN,NaN,...,47.107,135.799,18.691,332.967,13.329,285.860,285.860,NaN,35.571,85.207
4,12596130,CTI INDUSTRIES CORP,CTIB,2001-05-15,23.715,4.579,18.893,0.435,0.569,8.928,...,-0.196,12.208,-0.600,9.007,-0.138,7.534,9.203,0.327,-0.654,7.033
5,74372Q20,PROVALIS PLC,PVLS,None,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.165,2.645,2.149,23.141,2.149,9.587,23.306,0.000,-3.471,0.000
6,48669G10,KAZIA THERAPEUTICS LTD,KZIA,None,NaN,NaN,NaN,NaN,NaN,NaN,...,-8.576,5.561,-7.165,13.178,-7.018,21.754,21.754,0.089,-9.843,0.456
7,43858G10,HONG KONG TELE NETWORK LTD,HKTVY,None,NaN,NaN,NaN,NaN,NaN,NaN,...,31.448,121.284,20.047,82.247,18.584,15.258,50.799,0.014,20.019,2.330
8,75300930,RANDGOLD & EXPLORATION CO,RANGY,None,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.062,11.811,4.046,1.642,-0.266,2.704,2.704,0.207,-1.200,0.000
9,55914110,MAGIC LANTERN GROUP INC,GMLI,2001-05-15,4.760,0.000,3.218,0.299,NaN,0.022,...,-1.022,1.018,-1.404,0.690,-1.404,0.338,1.712,0.168,-1.236,1.435


In [62]:
# Eliminate all NaN data
merged_fin_df_clean = merged_fin_df.copy()

for column in merged_fin_df_clean.columns:
    if column not in ['conm', 'tic']:
        merged_fin_df_clean = merged_fin_df_clean.loc[merged_fin_df_clean[column].notnull()]
        
merged_fin_df_clean.reset_index(drop=True, inplace=True)

In [63]:
merged_fin_df_clean

,cusip,conm,tic,rdq_1,atq_1,dlttq_1,ltq_1,cheq_1,oibdpq_1,ppentq_1,...,oibdpq_12,ppentq_12,piq_12,revtq_12,niq_12,cogsq_12,xoprq_12,xintq_12,oiadpq_12,dlcq_12
0,12596130,CTI INDUSTRIES CORP,CTIB,2001-05-15,23.715,4.579,18.893,0.435,0.569,8.928,...,-0.196,12.208,-0.600,9.007,-0.138,7.534,9.203,0.327,-0.654,7.033
1,68555930,ORBIT INTERNATIONAL CORP,ORBT,2001-05-10,12.682,0.369,6.049,1.421,-0.072,0.238,...,0.315,0.164,0.321,4.078,0.321,2.229,3.763,0.000,0.297,0.149
2,97425910,WINMILL & CO INC -CL A,WNMLA,2001-05-15,7.961,0.000,0.154,3.427,-0.138,0.051,...,0.147,0.065,0.131,0.490,0.074,0.343,0.343,0.000,0.131,0.000
3,61690020,MORGANS FOODS INC,MRFD,2001-07-05,61.311,51.122,61.909,6.267,1.995,39.847,...,1.412,33.891,-1.538,22.865,-1.538,19.738,21.453,1.380,0.302,2.801
4,92823T10,VIRNETX HOLDING CORP,VHC,2001-05-14,0.425,0.000,0.289,0.375,0.095,0.012,...,0.023,0.000,0.022,0.068,0.020,0.000,0.045,0.000,0.023,0.000
5,87160C10,SYNERGX SYSTEMS INC,SYNX,2001-02-13,8.146,1.134,3.734,0.188,0.176,0.365,...,0.335,0.418,0.261,5.730,0.156,3.769,5.395,0.016,0.312,0.088
6,23936010,DAWSON GEOPHYSICAL CO,DWSN,2001-01-23,46.960,0.000,1.692,12.226,-1.154,27.073,...,1.660,20.780,0.609,14.695,0.557,12.389,13.035,0.000,0.527,0.000
7,88367150,THERMOVIEW INDUSTRIES INC,THVW,2001-05-09,71.143,16.143,25.806,1.284,-0.312,2.825,...,-0.363,2.676,-1.179,16.467,-1.180,8.920,16.830,0.728,-0.542,0.545
8,84780510,SPEIZMAN INDUSTRIES INC,SPZN,2000-11-15,58.967,16.314,39.280,0.000,-1.983,7.806,...,0.171,6.092,-0.410,14.552,-0.383,12.052,14.381,0.398,-0.012,8.520
9,67463110,OCEAN BIO-CHEM INC,OBCI,2001-05-11,15.166,3.832,10.827,0.025,-0.065,6.110,...,0.414,7.633,0.170,5.584,0.112,4.101,5.170,0.073,0.234,5.449


In [64]:
merged_fin_df_clean.to_csv("merged_fin_df_clean.csv")

# Monthly Data

In [34]:
# Iterate through all months and pull data

# Initialize start date
start_date = first_trading_day(pd.to_datetime('1-2-' + str(year))) 

# Initialize merged_df 
merged_monthly_df = smallcap_df[['cusip']].copy()

for m in range(12 * window_size):
    print("Month: %d" % m)
    
    # Initialize time range for each month
    start_date_timestamp = pd.to_datetime(daily_start_date)
    date = [start_date_timestamp + MonthEnd(m+1) - pd.Timedelta(str(10) + ' days'), start_date_timestamp + MonthEnd(m+1)] 
    date_str = [date[0].strftime('%Y-%m-%d'),date[1].strftime('%Y-%m-%d')]
    
    # SQL statement
    statement = "select all cusip, vol, ret, bid, ask, shrout, prc from crspq.msf where date > '{}' and date <= '{}'".format(date_str[0], date_str[1])
    
    # Pull data for each month
    monthly_data_df = db.raw_sql(statement)
    
    # Create label mapping to rename column labels
    label_map = {x : x + "_" + str(m + 1) for x in monthly_data_df.columns if x not in ['cusip']}
    
    # Use label mapping to rename column labels
    monthly_data_df.rename(columns=label_map, inplace=True)
    
    # Merge dataframe according to values in 'cusip' column
    merged_monthly_df = pd.merge(merged_monthly_df, monthly_data_df, how='inner', on=['cusip'])
        

Month: 0
Month: 1
Month: 2
Month: 3
Month: 4
Month: 5
Month: 6
Month: 7
Month: 8
Month: 9
Month: 10
Month: 11
Month: 12
Month: 13
Month: 14
Month: 15
Month: 16
Month: 17
Month: 18
Month: 19
Month: 20
Month: 21
Month: 22
Month: 23
Month: 24
Month: 25
Month: 26
Month: 27
Month: 28
Month: 29
Month: 30
Month: 31
Month: 32
Month: 33
Month: 34
Month: 35


In [36]:
merged_monthly_df

,cusip,vol_1,ret_1,bid_1,ask_1,shrout_1,prc_1,vol_2,ret_2,bid_2,...,bid_35,ask_35,shrout_35,prc_35,vol_36,ret_36,bid_36,ask_36,shrout_36,prc_36
0,23379840,143.0,-0.086758,1.900000,2.100000,31.0,-2.000000,120.0,0.100000,2.100000,...,45.200001,47.500000,16.0,-46.349998,46.0,0.014024,46.000000,48.000000,15.0,-47.000000
1,64378H10,4247.0,0.075000,0.380000,0.430000,40911.0,0.430000,1985.0,-0.209302,0.320000,...,1.640000,1.650000,47283.0,1.640000,58130.0,-0.140244,1.410000,1.420000,49322.0,1.410000
2,04940820,545.0,0.066667,4.760000,4.800000,85.0,4.800000,189.0,-0.054167,4.260000,...,3.170000,3.500000,202.0,3.500000,130.0,0.198914,3.800000,4.050000,190.0,4.050000
3,43123220,58.0,0.163333,1.500000,1.990000,277.0,-1.745000,50.0,0.146132,1.900000,...,12.750000,12.990000,277.0,12.750000,5716.0,0.073374,13.010000,13.040000,277.0,13.010000
4,26928220,3.0,-0.041667,0.750000,0.860000,613.0,-0.805000,49.0,0.397516,1.050000,...,1.950000,2.150000,858.0,-2.050000,71.0,-0.034146,1.940000,2.030000,858.0,1.980000
5,12596130,624.0,-0.130991,5.390000,5.490000,1910.0,-5.440000,447.0,-0.099265,4.850000,...,2.860000,2.930000,2036.0,2.930000,6866.0,-0.006826,2.790000,2.920000,2036.0,2.910000
6,48669G10,11986.0,-0.006380,9.440000,9.600000,3578.0,9.500000,5387.0,-0.025368,9.150000,...,20.299999,20.330000,7959.0,20.299999,5182.0,-0.104926,18.070000,18.170000,7966.0,18.170000
7,43858G10,62.0,0.135484,3.400000,3.640000,393.0,-3.520000,103.0,0.014205,3.570000,...,1.640000,1.680000,2275.0,1.640000,3821.0,-0.162988,1.370000,1.410000,2141.0,1.372700
8,04538V10,5203.0,0.005882,1.650000,1.840000,3269.0,1.710000,623.0,-0.011696,1.600000,...,2.500000,2.600000,3491.0,2.550000,341.0,0.000000,2.500000,2.600000,3491.0,-2.550000
9,15060220,93.0,0.288750,4.950000,5.360000,694.0,-5.155000,66.0,0.028128,5.030000,...,13.840000,13.850000,28509.0,13.750000,17575.0,0.023273,14.030000,14.040000,28509.0,14.070000


In [37]:
# Eliminate all NaN data
merged_monthly_df_clean = merged_monthly_df.copy()

for column in merged_monthly_df_clean.columns:
    merged_monthly_df_clean = merged_monthly_df_clean.loc[merged_monthly_df_clean[column].notnull()]
        
merged_monthly_df_clean.reset_index(drop=True, inplace=True)

In [38]:
merged_monthly_df_clean

,cusip,vol_1,ret_1,bid_1,ask_1,shrout_1,prc_1,vol_2,ret_2,bid_2,...,bid_35,ask_35,shrout_35,prc_35,vol_36,ret_36,bid_36,ask_36,shrout_36,prc_36
0,23379840,143.0,-0.086758,1.900000,2.100000,31.0,-2.000000,120.0,0.100000,2.100000,...,45.200001,47.500000,16.0,-46.349998,46.0,0.014024,46.000000,48.000000,15.0,-47.000000
1,64378H10,4247.0,0.075000,0.380000,0.430000,40911.0,0.430000,1985.0,-0.209302,0.320000,...,1.640000,1.650000,47283.0,1.640000,58130.0,-0.140244,1.410000,1.420000,49322.0,1.410000
2,04940820,545.0,0.066667,4.760000,4.800000,85.0,4.800000,189.0,-0.054167,4.260000,...,3.170000,3.500000,202.0,3.500000,130.0,0.198914,3.800000,4.050000,190.0,4.050000
3,43123220,58.0,0.163333,1.500000,1.990000,277.0,-1.745000,50.0,0.146132,1.900000,...,12.750000,12.990000,277.0,12.750000,5716.0,0.073374,13.010000,13.040000,277.0,13.010000
4,26928220,3.0,-0.041667,0.750000,0.860000,613.0,-0.805000,49.0,0.397516,1.050000,...,1.950000,2.150000,858.0,-2.050000,71.0,-0.034146,1.940000,2.030000,858.0,1.980000
5,12596130,624.0,-0.130991,5.390000,5.490000,1910.0,-5.440000,447.0,-0.099265,4.850000,...,2.860000,2.930000,2036.0,2.930000,6866.0,-0.006826,2.790000,2.920000,2036.0,2.910000
6,48669G10,11986.0,-0.006380,9.440000,9.600000,3578.0,9.500000,5387.0,-0.025368,9.150000,...,20.299999,20.330000,7959.0,20.299999,5182.0,-0.104926,18.070000,18.170000,7966.0,18.170000
7,43858G10,62.0,0.135484,3.400000,3.640000,393.0,-3.520000,103.0,0.014205,3.570000,...,1.640000,1.680000,2275.0,1.640000,3821.0,-0.162988,1.370000,1.410000,2141.0,1.372700
8,04538V10,5203.0,0.005882,1.650000,1.840000,3269.0,1.710000,623.0,-0.011696,1.600000,...,2.500000,2.600000,3491.0,2.550000,341.0,0.000000,2.500000,2.600000,3491.0,-2.550000
9,15060220,93.0,0.288750,4.950000,5.360000,694.0,-5.155000,66.0,0.028128,5.030000,...,13.840000,13.850000,28509.0,13.750000,17575.0,0.023273,14.030000,14.040000,28509.0,14.070000


In [39]:
merged_monthly_df_clean.to_csv("merged_monthly_df_clean.csv")

# Select common companies in financial and monthly dataframe based on common cusip

In [65]:
# Get cusip index from two dataframes

fin_cusip_index = pd.Index(merged_fin_df_clean['cusip'])
monthly_cusip_index = pd.Index(merged_monthly_df_clean['cusip'])

print(fin_cusip_index)
print(fin_cusip_index.shape)
print(monthly_cusip_index)
print(monthly_cusip_index.shape)

Index(['12596130', '68555930', '97425910', '61690020', '92823T10', '87160C10',
       '23936010', '88367150', '84780510', '67463110',
       ...
       '04362P20', '91913Y10', 'H2906T10', '12653C10', '50544710', '27826510',
       '31190010', '17038810', '20366810', '12705510'],
      dtype='object', name='cusip', length=1688)
(1688,)
Index(['23379840', '64378H10', '04940820', '43123220', '26928220', '12596130',
       '48669G10', '43858G10', '04538V10', '15060220',
       ...
       '74790650', '50544710', '73172K10', '27826510', '30249130', '31190010',
       '80004C10', '17038810', '20366810', '12705510'],
      dtype='object', name='cusip', length=4015)
(4015,)


In [66]:
# Intersection on cusip index

common_cusip_index = fin_cusip_index.intersection(monthly_cusip_index)

common_cusip_index

Index(['12596130', '68555930', '87160C10', '67463110', '00256410', '23337740',
       '18725U10', '56660510', '04316810', '41766040',
       ...
       '41387510', '91913Y10', 'H2906T10', '12653C10', '50544710', '27826510',
       '31190010', '17038810', '20366810', '12705510'],
      dtype='object', name='cusip', length=1361)

In [67]:
# Get filtered common financial and monthly dataframes

joint_fin_df = merged_fin_df_clean.loc[merged_fin_df_clean['cusip'].isin(common_cusip_index)]
joint_fin_df.reset_index(drop=True, inplace=True)
joint_fin_df

,cusip,conm,tic,rdq_1,atq_1,dlttq_1,ltq_1,cheq_1,oibdpq_1,ppentq_1,...,oibdpq_12,ppentq_12,piq_12,revtq_12,niq_12,cogsq_12,xoprq_12,xintq_12,oiadpq_12,dlcq_12
0,12596130,CTI INDUSTRIES CORP,CTIB,2001-05-15,23.715,4.579,18.893,0.435,0.569,8.928,...,-0.196,12.208,-0.600,9.007,-0.138,7.534,9.203,0.327,-0.654,7.033
1,68555930,ORBIT INTERNATIONAL CORP,ORBT,2001-05-10,12.682,0.369,6.049,1.421,-0.072,0.238,...,0.315,0.164,0.321,4.078,0.321,2.229,3.763,0.000,0.297,0.149
2,87160C10,SYNERGX SYSTEMS INC,SYNX,2001-02-13,8.146,1.134,3.734,0.188,0.176,0.365,...,0.335,0.418,0.261,5.730,0.156,3.769,5.395,0.016,0.312,0.088
3,67463110,OCEAN BIO-CHEM INC,OBCI,2001-05-11,15.166,3.832,10.827,0.025,-0.065,6.110,...,0.414,7.633,0.170,5.584,0.112,4.101,5.170,0.073,0.234,5.449
4,00256410,ABATIX CORP,ABIX,2001-05-07,15.810,0.000,8.983,0.028,0.598,0.665,...,-0.065,1.294,-0.219,10.909,-0.153,7.810,10.974,0.044,-0.176,4.523
5,23337740,DXP ENTERPRISES INC,DXPE,2001-05-14,64.662,24.593,55.662,0.456,1.524,9.565,...,1.181,7.395,0.693,35.093,0.504,25.604,33.912,0.230,0.908,1.474
6,18725U10,CLINICAL DATA INC,CLDA,2001-08-10,7.701,0.017,3.670,1.038,0.179,0.760,...,1.768,1.895,1.383,15.740,1.443,10.091,13.972,0.056,1.466,1.872
7,56660510,MARGO CARIBE INC,MRGO,2001-05-01,9.318,0.240,4.008,1.191,0.206,1.683,...,-1.025,2.293,-1.112,1.909,-1.112,1.733,2.934,0.018,-1.073,2.829
8,04316810,ARTS WAY MFG INC,ARTW,2001-04-09,10.780,0.326,7.012,0.012,0.016,1.906,...,0.287,1.019,0.163,2.865,0.966,1.873,2.578,0.067,0.220,0.179
9,41766040,HARVEY ELECTRONICS INC,HRVEQ,2001-03-13,11.488,0.036,6.262,0.007,0.777,2.694,...,0.177,2.780,-0.153,9.664,-0.098,5.551,9.487,0.080,-0.038,0.000


In [68]:
joint_monthly_df = merged_monthly_df_clean.loc[merged_monthly_df_clean['cusip'].isin(common_cusip_index)]
joint_monthly_df.reset_index(drop=True, inplace=True)
joint_monthly_df

,cusip,vol_1,ret_1,bid_1,ask_1,shrout_1,prc_1,vol_2,ret_2,bid_2,...,bid_35,ask_35,shrout_35,prc_35,vol_36,ret_36,bid_36,ask_36,shrout_36,prc_36
0,12596130,624.0,-0.130991,5.390000,5.490000,1910.0,-5.440000,447.0,-0.099265,4.850000,...,2.860000,2.930000,2036.0,2.930000,6866.0,-0.006826,2.790000,2.920000,2036.0,2.910000
1,68555930,2126.0,0.032967,4.390000,4.640000,2112.0,4.700000,735.0,-0.148936,4.000000,...,13.990000,14.080000,4550.0,13.960000,4824.0,-0.107450,12.400000,12.460000,4575.0,12.460000
2,87160C10,748.0,0.406250,1.710000,1.800000,1874.0,1.800000,636.0,0.000000,1.810000,...,1.820000,1.860000,5192.0,1.830000,7988.0,-0.010929,1.820000,1.830000,5211.0,1.810000
3,67463110,383.0,0.272109,1.790000,1.950000,4806.0,-1.870000,250.0,-0.021390,1.640000,...,0.770000,0.800000,5690.0,0.770000,28636.0,0.315195,1.010000,1.030000,5690.0,1.012700
4,00256410,255.0,-0.041429,6.570000,6.950000,1711.0,6.710000,338.0,0.207154,7.060000,...,12.440000,12.550000,1711.0,12.540000,4670.0,-0.109250,11.080000,11.270000,1711.0,11.170000
5,23337740,413.0,-0.036697,1.030000,1.070000,4072.0,-1.050000,199.0,-0.061905,0.950000,...,16.250000,16.330000,4736.0,16.340000,23699.0,0.052632,17.180000,17.200001,5055.0,17.200001
6,18725U10,62.0,0.041026,3.930000,4.190000,1855.0,-4.060000,94.0,-0.064039,3.790000,...,21.299999,21.490000,6704.0,21.490000,2928.0,-0.185668,17.500000,17.570000,7968.0,17.500000
7,56660510,62.0,-0.098143,3.200000,3.600000,2080.0,-3.400000,105.0,0.294118,4.200000,...,5.500000,5.860000,2786.0,5.500000,505.0,0.254545,6.850000,7.290000,2793.0,6.900000
8,04316810,21.0,0.215517,3.100000,3.950000,1938.0,-3.525000,524.0,-0.004255,3.410000,...,4.850000,4.920000,1963.0,4.850000,713.0,0.092784,5.240000,5.430000,1963.0,5.300000
9,41766040,971.0,-0.125000,1.040000,1.060000,3325.0,1.050000,519.0,0.038095,1.040000,...,0.820000,0.840000,3509.0,0.840000,3963.0,-0.178571,0.660000,0.690000,3509.0,0.690000


In [77]:
directory = "window_data"

In [78]:
os.path.isdir(directory)

False

In [79]:
os.mkdir(directory)

In [80]:
fin_path = os.path.join(directory, "fin_%d.csv" % year)

fin_path

'window_data/fin_2001.csv'

In [82]:
if not os.path.isdir(directory):
    os.mkdir(directory)

joint_fin_df.to_csv(fin_path)